In [116]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [117]:
#today, we will convert our data for sex into a numerical value so that the computer can read it better (it doesn't know how to read in words quite yet)
#we will also run our data without depending on drop.na()

In [118]:
filepath = '/kaggle/input/titanic/train.csv' #creating our filepath
titanic_data = pd.read_csv(filepath)


In [119]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [120]:
titanic_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [121]:
y = titanic_data.Survived #we're tryna find the survival rate; our dependent variables
features = ['Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']
X = titanic_data[features] #our independent variables

#we're just going to ignore cabin class for now because so much of the data is missing 

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 0)

In [123]:
#cols_with_missing = [col for col in X_train.columns
                    #if X_train[col].isnull().any()]

#Drop columns in training and validation data that have missing data in them
#reduced_X_train = X_train.drop(cols_with_missing, axis = 1)
#reduced_X_valid = X_valid.drop(cols_with_missing, axis = 1)

In [124]:
#do DTC
#from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier(random_state = 0)
#model.fit(reduced_X_train, y_train)

In [125]:
#from sklearn.metrics import classification_report
#y_pred = model.predict(reduced_X_valid)
#print (classification_report(y_valid, y_pred))

In [126]:
#why is this precision and recall more accurate? Because we used more data.

In [127]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
OH_X_train = pd.DataFrame(OH_encoder.fit_transform(X_train[['Sex']]))
OH_X_valid = pd.DataFrame(OH_encoder.transform(X_valid[['Sex']]))

OH_X_train.index = X_train.index
OH_X_valid.index = X_valid.index

OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

total_X_train = pd.concat([X_train, OH_X_train], axis = 1)
total_X_valid = pd.concat([X_valid, OH_X_valid], axis = 1)

total_X_train.drop('Sex', axis = 1, inplace = True)
total_X_valid.drop('Sex', axis = 1, inplace = True)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [128]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer(strategy = 'mean')

total_X_train_imputed = my_imputer.fit_transform(total_X_train)
total_X_valid_imputed = my_imputer.transform(total_X_valid)

model = DecisionTreeClassifier(random_state = 0)
model.fit(total_X_train_imputed, y_train)

DecisionTreeClassifier(random_state=0)

In [130]:
from sklearn.metrics import classification_report
y_pred = model.predict(total_X_train_imputed)
print (classification_report(y_valid, y_pred_))

ValueError: Found input variables with inconsistent numbers of samples: [268, 623]